In [21]:
# Pythonの基本ライブラリ
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# spotifyのAPIを利用
import spotipy

# Jupyter上にHTMLを表示する
from IPython.display import HTML

In [2]:
# Spotify APIの準備

# APIキーをファイルから取得
f = open('../secret/clientsecret_spotify', 'r')
client_secret = f.read()
f.close()
client_id, client_secret = client_secret.split('\n')[0], client_secret.split('\n')[1]
    
client_credentials_manager = spotipy.oauth2.SpotifyClientCredentials(client_id, client_secret)
spotify = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [3]:
df = pd.read_csv('dataset/streams.csv', header=0)
df.head()

,Artists,SongTitle,VideoID,StartTime
0,星街すいせい,破滅の純情,rbkjVA6Fjh0,4872
1,星街すいせい,コワレヤスキ,rbkjVA6Fjh0,5743
2,"星街すいせい,常闇トワ",ライオン,rbkjVA6Fjh0,6763
3,星街すいせい,天球、彗星は夜を跨いで,FZnG1t34wCs,95
4,星街すいせい,Neo stream,FZnG1t34wCs,568


* q="track:~"で検索すると明らかにヒットしそうな情報も出てこないことがある
    * 範囲は広くなるがq="直接タイトル"の方を採用する
* アイマスやアンスタなどサブスクに載っていない曲はどうにもならないためカット
    * これらの動画からは特徴量の抽出は諦めつつ、アーティスト名だけは埋めておきたい
    * ただし、後程曲の長さを手動で入力する必要があるため、その時で良い
* 検索された中で、マッチング率が高いものに絞ると効率が良さそう
    * しかし、そもそもWikiの表記揺れもあり確信度がそれほど高くないので一旦人力での確認とする
* 欲しい情報は楽曲ID、アーティスト名、アーティストID、アーティストの画像URL、曲の長さ、ジャンル、など？
    * 特徴量とかはちょっと違った性質になるため後で追加でも良いかも（IDさえ控えておけば問題なし）
* inputで受け取る形でひとまずはやってみる
    * 補助情報としては動画へのリンクくらいで十分か
* 将来的にはキーボードショートカットに対応したアノテーションツールが欲しい

In [27]:
# 補助情報としてリンクを作成
def make_url(video_id, time):
    return "https://www.youtube.com/watch?v="+video_id+"&t="+time+"s"

def output_html(video_id, time):
    html = '<div style="float:left;">'
    html += ('<img src="http://img.youtube.com/vi/'+video_id+'/sddefault.jpg "alt="取得できませんでした" width="80">')
    url = make_url(video_id, time)
    html += '   '
    html += ('<a href="'+url+'">'+url+'</a><br>') 
    html += '</div>'
    return html

In [78]:
# 過去のデータを保存する場合
diff = True
already_ids = []
if diff:
    output_dir_path = 'dataset/streams_1.csv'
    already_song_details = pd.read_csv(output_dir_path, header=0)
    already_ids = already_song_details.index.tolist()

In [ ]:
# APIを使って情報を取り出す
# 一気に600件やると骨が折れるので適宜保存するようにしておく
# あとは結合して保存できるようにすれば、、、特徴量やendtime、コラボ属性などの付与などを後で行えばひとまずは完成？？？
# サムネだけならYoutube APIも不要か

song_details = []
output_dir_path = 'dataset/streams_1.csv'
for i, row in df.iterrows():
    save = True
    print(i+1, "/", df.shape[0])
    # 既に保存済みの場合は削除
    if i in already_ids:
        continue
    
    song_title = row['SongTitle']
    # result = spotify.search(q="track:" + song_title, limit=10, type="track")
    result = spotify.search(q=song_title, limit=10, type='track')
    
    members = row['Artists']
    video_id = row['VideoID']
    start_time = row['StartTime']
    # 中間でHTMLの表示ができなかったので断念
    # HTML(output_html(video_id, start_time))
    print('検索ワード', song_title, '&', make_url(video_id, start_time))
    
    items = result['tracks']['items']
    # 検索結果を取り出す（この中から近いものを取り出さなくてはならない）
    for j, item in enumerate(items):
        track_name = item['name']
        track_id = item['id']
        track_url = item['external_urls']['spotify']
        artist_name = item['artists'][0]['name'] # 複数いる場合もありそう
        artist_id = item['artists'][0]['id']
        print(j, ':', track_name, '&', artist_name, '&', track_url)
    
    # 参考にするデータを選択
    
    if len(items) == 0:
        print('検索結果が無かったためスキップします\n')
        save = False
    if save:
        try:
            index = input()
            index = int(index)
        except ValueError:
            print('数字以外の値なためスキップします\n')
            save = False
    if save:
        if index < 0 or index >= len(items):
            print('indexが指定外の値のためスキップします\n')
            save = False

    # 追加の情報を入れる
    # track: id, name, duration
    # artist: id, name, image_url, genres    
    if save:
        item = items[index]
        track_id = item['id']
        track_name = item['name']
        duration_ms = item['duration_ms']
        artist_id = item['artists'][0]['id']
        artist_name = item['artists'][0]['name'] # 複数いる場合もありそう

        artist = spotify.artist(artist_id)
        # imagesが存在しない場合もある
        artist_url = 'None'
        if len(artist['images']) > 0:
            artist_url = artist['images'][0]['url']
        # 空の場合もある
        artist_genres = artist['genres']
        artist_genres_string = 'None'
        if len(artist_genres) > 0:
            artist_genres_string = ','.join(artist_genres)
        print(track_name, '&', artist_name)
    else:
        track_id = 'None'
        track_name = 'None'
        duration_ms = 0
        artist_id = 'None'
        artist_name = 'None'
        artist_url = 'None'
        artist_genres_string = 'None'
    
    song_details.append([members, song_title, video_id, start_time, 
                         track_id, track_name, duration_ms, artist_id, artist_name, artist_url, artist_genres_string])
    song_details_pandas = pd.DataFrame(data=np.array(song_details), 
                                       columns=['Members', 'SongName', 'VideoID', 'StartTime',
                                                'TrackID', 'TrackName', 'Duration', 'ArtistID', 'ArtistName', 'ArtistURL', 'ArtistGenres'])
    
    if diff:
        song_details_tmp = pd.concat([already_song_details, song_details_pandas])
    else:
        song_details_tmp = song_details_pandas
    # 特に処理が遅くなるわけでも無さそうなので毎回保存する
    period = 1
    if i % period == 0:
        song_details_tmp.to_csv(output_dir_path, index=False)
    
    print()

1 / 671
2 / 671
3 / 671
4 / 671
5 / 671
6 / 671
7 / 671
8 / 671
9 / 671
10 / 671
11 / 671
12 / 671
13 / 671
14 / 671
検索ワード 文学少年の憂鬱 & https://www.youtube.com/watch?v=n0ieG2D_qaU&t=658s
0 : 文学少年の憂鬱 & Lyu:Lyu & https://open.spotify.com/track/50dc7FEQNEAnVfKCoNZtrI


In [77]:
# 全く自信がないのでどこかでダブルチェックしたい
output_dir_path = 'dataset/streams_1.csv'
song_details = pd.read_csv(output_dir_path, header=0)
song_details

,Members,SongName,VideoID,StartTime,TrackID,TrackName,Duration,ArtistID,ArtistName,ArtistURL,ArtistGenres
0,星街すいせい,破滅の純情,rbkjVA6Fjh0,4872,4Rs0bABgDMzAqAAd2P8ArD,Hametsu no Junjo,267400,5U8dcKsZ0mmeiUgbM0jvFC,Sheryl Nome starring May'n,https://i.scdn.co/image/ab67616d0000b27364c32c...,None
1,星街すいせい,コワレヤスキ,rbkjVA6Fjh0,5743,1TizL66jSoN48Hj8airJgr,コワレヤスキ,302453,7gKNDpetmXrzHZuLdo9tv8,Guilty Kiss,https://i.scdn.co/image/ab67616d0000b273def723...,love live
2,"星街すいせい,常闇トワ",ライオン,rbkjVA6Fjh0,6763,1ux8Y96cV5Qs0kdOCq4cZ2,Lion,303657,0JQH8OHvGdooprROP18Wg6,May'n,https://i.scdn.co/image/ab6761610000e5eb0d1c05...,"anime,j-poprock"
3,星街すいせい,天球、彗星は夜を跨いで,FZnG1t34wCs,95,0wTs0OV88H5YZrFX6zN4QJ,天球、彗星は夜を跨いで,256548,726WiFmWkohzodUxK3XjHX,Hoshimachi Suisei,https://i.scdn.co/image/ab6761610000e5ebee9358...,"hololive,japanese vtuber"
4,星街すいせい,Neo stream,FZnG1t34wCs,568,1WZ2n937qxjPuJAHzFGegK,NEO STREAM,270960,4b1IVV9meynYx65WpsxjbT,Walkure,https://i.scdn.co/image/ab67616d0000b273ab4899...,None
5,星街すいせい,Absolute 5,FZnG1t34wCs,863,76BBm1eodAEYc6Vcxcsjcw,Absolute 5,268000,4b1IVV9meynYx65WpsxjbT,Walkure,https://i.scdn.co/image/ab67616d0000b273ab4899...,None
6,星街すいせい,射手座☆午後九時don't be late,FZnG1t34wCs,1148,4WU7VzsNIEyXKQMyf2JfaB,射手座☆午後九時Don't be late,346893,5fruBgRiTeuJwkU5qvrdTq,シェリル・ノーム starring May'n,https://i.scdn.co/image/ab67616d0000b2730975d6...,macross
7,星街すいせい,共鳴世界の存在論,FZnG1t34wCs,1745,None,None,0,None,None,None,None
8,星街すいせい,Hotel moonside,FZnG1t34wCs,2018,None,None,0,None,None,None,None
9,星街すいせい,Daydream Warrior,FZnG1t34wCs,2384,449JyzwBG4L3IVHUf9zkZs,Daydream Warrior,275173,6zxQda06WxXX8GmCeYstwV,Aqours,https://i.scdn.co/image/ab6761610000e5ebc59827...,"anime,love live"
